We import what will serve us

In [2]:
from pandas import DataFrame, read_csv
import bigml.api
from bigml.api import BigML
import kaggle


## Prediction in validation set

We keep the best number : 3 

we train the ensemble in all the training test

In [3]:
api = BigML(project='project/5db160c5f80b1640d6000224')

In [5]:
source_full = api.create_source('kaggle/fulltrain3.csv')
# waiting for the source to be finished. Results will be stored in `source`
api.ok(source_full)
# step 3: creating a dataset from the previously created `source`
dataset_full = api.create_dataset(source_full)
# waiting for the dataset to be finished
api.ok(dataset_full) 
# step 5: creating an Ensemble
ensemble_full = api.create_ensemble(dataset_full)
# waiting for the ensemble to be finished
api.ok(ensemble_full)

True

Make prediction in the validation set 

In [8]:
# step 6: creating a source from the data in your local "data/test_iris.csv" file
test_source_validation = api.create_source("kaggle/validation3.csv")
# waiting for the source to be finished. Results will be stored in `source`
api.ok(test_source_validation)
# step 8: creating a dataset from the previously created `source`
test_dataset_validation = api.create_dataset(test_source_validation)
# waiting for the dataset to be finished
api.ok(test_dataset_validation)
# step 10: creating a batch prediction
batch_prediction_validation = api.create_batch_prediction(ensemble_full, test_dataset_validation,
                                               {"all_fields": True,
                                                 "header": True,
                                                "confidence": True,
                                                "probabilities": True})
# waiting for the batch_prediction to be finished
api.ok(batch_prediction_validation)
# downloading the results to your computer
api.download_batch_prediction(batch_prediction_validation,
                              filename='kaggle/kaggle_predictions3.csv')

'kaggle/kaggle_predictions3.csv'

We recover the prediction

In [9]:
prediction=read_csv('kaggle/kaggle_predictions3.csv',index_col=False)
print(list(prediction))

['field1', 'RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'DisposableIncome', 'BalancedIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfTimes90DaysLate', 'WeightedOfLatePayment', 'NumberOfDependents', 'SeriousDlqin2yrs', 'SeriousDlqin2yrs.1', 'confidence', '0 probability', '1 probability']


We rename the columns for rendering

In [10]:
kaggle_prediction=DataFrame()
kaggle_prediction['Id']=prediction['field1']
kaggle_prediction['Probability']=prediction['1 probability']

We check the output

In [11]:
kaggle_prediction.head()

,Id,Probability
0,1,0.05602
1,2,0.04954
2,3,0.02553
3,4,0.05403
4,5,0.13165


Render preparation and sending for evaluation by Kaggle

In [12]:
kaggle_prediction_file="kaggleprediction_3.csv"
kaggle_prediction.to_csv(kaggle_prediction_file,index=False)
kaggle.api.competition_submit(kaggle_prediction_file, "Prediction Bigml", "GiveMeSomeCredit")

100%|██████████| 1.66M/1.66M [00:06<00:00, 278kB/s] 


Successfully submitted to Give Me Some Credit

Maud SONET